In [ ]:
import pandas as pd
import csv
import numpy as np
import plotly.graph_objects as go

In [ ]:
df = pd.DataFrame(columns=['day','factor','required','delivery','percents','num_vehicle','carbon_emission(kg)'])
df.to_csv('TOU_P120.csv')

In [ ]:
path = 'differentfactor.csv'
with open(path,'a+') as f:
    csv_write = csv.writer(f)
    data = ['0','0','0','100','100','1','-1','2']
    csv_write.writerow(data)

In [ ]:
pd.read_csv(path,index_col=0)

# Plot

In [ ]:
d1 = pd.read_csv('differentfactor.csv', index_col=0)
d2 = pd.read_csv('EDF.csv', index_col=0)
mpc_carbon = []
mpc_delivery = []
for factor in [0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]:
    dptmp = d1[d1['factor']==factor]
    #dptmp = dptmp[dptmp['num_vehicle'] < 20]
    print(len(dptmp))
    mpc_delivery.append(100-np.sum(dptmp['delivery'])/np.sum(dptmp['required'])*100)
    mpc_carbon.append(np.mean(dptmp['carbon_emission(kg)'])*20)
mpc_carbon[7] = mpc_carbon[-1]
mpc_delivery[7] =mpc_delivery[-1]
d2tmp = d2.iloc[:20]
#d2tmp = d2tmp[d2tmp['num_vehicle'] < 20]
print(len(d2tmp))
edf_carbon = np.mean(d2tmp['carbon_emission(kg)']) * 20
edf_delivery = 100-np.sum(d2tmp['delivery']) / np.sum(d2tmp['required']) * 100

In [ ]:
np.array([100 for _ in range(13)]) - np.array(mpc_delivery)

In [133]:
fig = go.Figure()
factors = [0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]
n = len(factors)


fig.add_trace(go.Scatter(x=factors, y=mpc_carbon, name='Carbon emission (kg)', line=dict(color='#ef857e', width=3)))
fig.add_trace(go.Scatter(x=factors, y=mpc_delivery, yaxis='y2',name='Delivery energy (%)', line=dict(color='#8cc1e6', width=3)))
#fig.add_trace(go.Scatter(x=[factors[2]], y=[mpc_carbon[2]],mode='markers',marker_symbol='star',showlegend = False,marker_color="#ef857e", marker_size=15))
#fig.add_trace(go.Scatter(x=[factors[2]], y=[mpc_delivery[2]],yaxis='y2',mode='markers',marker_color="#8cc1e6",marker_symbol='star',showlegend = False,marker_size=15))

fig.add_trace(go.Scatter(x=factors, y=[edf_carbon for _ in range(n)], name='Baseline Carbon emission (kg)', line=dict(color='#ef857e', width=3, dash='dash')))
fig.add_trace(go.Scatter(x=factors, y=[edf_delivery for _ in range(n)], yaxis='y2',name='Baseline Delivery energy (%)', line=dict(color='#8cc1e6', width=3, dash='dash')))

fig.update(layout={
    'xaxis':{'title':'Balanced factor', 'showline':True},
    'yaxis':{'title': 'Carbon emission(kg)', 'color':'#ef857e'},
    'yaxis2':{'title': 'Energy loss (%)','range':[0,101],'anchor':'x','overlaying':'y','side':'right','color':'#8cc1e6'},
    "font": {"size": 20,},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':900,
    'height':500,
    #  'xaxis':{ 'showgrid': False},
    # 'yaxis': {   'showgrid': True} 
})
fig.show()

In [ ]:
fig.write_image("image/different_factor_3.pdf")

In [ ]:
fig = go.Figure()
factors = ['1','10','100','1000']
mpc_mean = [np.mean(d1[d1['factor']==factor]['percents']) for factor in [1,10,100,1000]]
mpc_20 = [np.quantile(d1[d1['factor']==factor]['percents'],0.2) for factor in [1,10,100,1000]]
mpc_80 = [np.quantile(d1[d1['factor']==factor]['percents'],0.8) for factor in [1,10,100,1000]]
edf_20 = np.quantile(d2['percents'],0.2)
edf_80 = np.quantile(d2['percents'],0.8)
edf_mean = np.mean(d2['percents'])
fig.add_trace(go.Scatter(x=factors, y=mpc_mean, name='MPC(average)', line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=factors, y=mpc_20, name='MPC(20%)', line=dict(color='firebrick', width=5,dash='dash')))
fig.add_trace(go.Scatter(x=factors, y=mpc_80, name='MPC(80%)', line=dict(color='firebrick', width=4,dash='dash')))

fig.add_trace(go.Scatter(x=factors, y=[edf_mean for _ in range(4)], name='EDF(average)', line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=factors, y=[edf_20 for _ in range(4)], name='EDF(20%)', line=dict(color='royalblue', width=4,dash='dash')))
fig.add_trace(go.Scatter(x=factors, y=[edf_80 for _ in range(4)], name='EDF(80%)', line=dict(color='royalblue', width=4,dash='dash')))

fig.update(layout={
    'xaxis':{'title':'Balanced factor', 'showline':True},
    'yaxis':{'title': 'Energy delivery (percents)','range':[0,100]},
    "font": {"size": 20},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':900,
    'height':500
})
fig.show()

In [ ]:
num_of_vehicles = np.unique(d1['num_vehicle'])
num_of_vehicles

# Plot $\lambda$ with different arrived vehicles

In [ ]:
d1 = pd.read_csv('MPC_offline.csv', index_col=0)
d11 = d1[d1['factor']==1]
d12 = d1[d1['factor']==1.5]
d13 = d1[d1['factor']==5]
d2 = pd.read_csv('EDF.csv', index_col=0)
num_of_vehicles = np.unique(d1['num_vehicle'])
offline_c = []
offline_e = []
offline_c2 = []
offline_e2 = []
offline_c3 = []
offline_e3 = []
edf_c = []
edf_e = []
for v in num_of_vehicles[::2]:
    tmpd1 = d11[d11['num_vehicle']==v]
    tmpd12 = d12[d12['num_vehicle']==v]
    tmpd13 = d13[d13['num_vehicle']==v]
    tmpd2 = d2[d2['num_vehicle']==v]
    offline_c.append(np.mean(tmpd1['carbon_emission(kg)']))
    edf_c.append(np.mean(tmpd2['carbon_emission(kg)']))
    offline_e += [np.sum(tmpd1['delivery']) / np.sum(tmpd1['required']) * 100]
    edf_e += [np.sum(tmpd2['delivery']) / np.sum(tmpd2['required']) * 100]
    offline_c2.append(np.mean(tmpd12['carbon_emission(kg)']))
    offline_e2 += [np.sum(tmpd12['delivery']) / np.sum(tmpd12['required']) * 100]
    offline_c3.append(np.mean(tmpd13['carbon_emission(kg)']))
    offline_e3 += [np.sum(tmpd13['delivery']) / np.sum(tmpd13['required']) * 100]

In [ ]:
offline_e2

In [132]:
from plotly.subplots import make_subplots
x = ['carbon-aware','baseline']
fig = make_subplots(rows=1, cols=2, subplot_titles=("factor=1","factor=1.5","factor=5"),specs=[[{"secondary_y": True}, {"secondary_y": True}]])

x = num_of_vehicles[::2]
#fig.add_trace(go.Scatter(x=x, y=offline_c, name='factor=1', line=dict(color='#ef857e', width=2)))
fig.add_trace(go.Scatter(x=x[:20], y=offline_c[:20], name='Carbon aware: Carbon emission(kg)', line=dict(color='#ef857e', width=2)),row=1,col=1)
fig.add_trace(go.Scatter(x=x[:20], y=offline_e[:20], name='Carbon aware: Energy delivery(%)', line=dict(color='#8cc1e6', width=2)),row=1,col=1,secondary_y=True)

fig.add_trace(go.Scatter(x=x[:20], y=edf_c[:20], name='Baseline: Carbon emission(kg)', line=dict(color='#ef857e', width=2, dash='dash')),row=1,col=1)
fig.add_trace(go.Scatter(x=x[:20], y=edf_e[:20], yaxis='y2',name='Baseline: Energy delivery(%)', line=dict(color='#8cc1e6', width=2,dash='dash')),row=1,col=1,secondary_y=True)

fig.add_trace(go.Scatter(x=x, y=offline_c2, showlegend=False, line=dict(color='#ef857e', width=2)),row=1,col=2)
fig.add_trace(go.Scatter(x=x, y=offline_e2, showlegend=False, line=dict(color='#8cc1e6', width=2)),row=1,col=2,secondary_y=True)

fig.add_trace(go.Scatter(x=x, y=edf_c, showlegend=False, line=dict(color='#ef857e', width=2, dash='dash')),row=1,col=2)
fig.add_trace(go.Scatter(x=x, y=edf_e, yaxis='y2',showlegend=False, line=dict(color='#8cc1e6', width=2,dash='dash')),row=1,col=2,secondary_y=True)

# fig.add_trace(go.Scatter(x=x, y=offline_c3, showlegend=False, line=dict(color='#ef857e', width=2)),row=1,col=3)
# fig.add_trace(go.Scatter(x=x, y=offline_e3, showlegend=False, line=dict(color='#8cc1e6', width=2)),row=1,col=3,secondary_y=True)

# fig.add_trace(go.Scatter(x=x, y=edf_c, showlegend=False, line=dict(color='#ef857e', width=2, dash='dash')),row=1,col=3)
# fig.add_trace(go.Scatter(x=x, y=edf_e, yaxis='y2',showlegend=False, line=dict(color='#8cc1e6', width=2,dash='dash')),row=1,col=3,secondary_y=True)
fig.add_vrect(
    x0=0, x1=20,
    fillcolor='#a8ecd9', opacity=0.5,
    layer="below", line_width=0,row=1,col=1
)
fig.add_vrect(
    x0=14, x1=36,
    fillcolor='#a8ecd9', opacity=0.5,
    layer="below", line_width=0,row=1,col=2
)

fig.update(layout={
    'xaxis':{'title':'Num of vehicles arrived', 'showline':True},
    'yaxis':{'title': 'Carbon emission(kg)'},
    'yaxis2':{'title': 'Energy delivered  (%)','range':[0,101],'anchor':'x','overlaying':'y','side':'right'},
    'yaxis4':{'range':[1,101]},
    'yaxis6':{'range':[1,101]},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':1000,
    'height':500
})
fig.show()

In [ ]:
fig.write_image("image/different_factor_4.pdf")

# Plot comparison baselines

In [ ]:
d1 = pd.read_csv('MPC_offline.csv', index_col=0)
d11 = d1[d1['factor']==1]
d12 = d1[d1['factor']==1.5]
d12 = d1[d1['factor']==1.5]
d2 = pd.read_csv('EDF_P60.csv', index_col=0)
d3 = pd.read_csv('ES_P60.csv', index_col=0)



x = ['Offline','EDF','ES']
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(name='Total carbon_emission(kg)', x=x, y=[np.sum(d['carbon_emission(kg)']) for d in [d11,d2,d3]],marker_color='#ef857e'),secondary_y=False)
fig.add_trace(go.Bar(name='Energy delivery(%)', x=x, y=[np.sum(d['delivery'])/np.sum(d['required'])*100 for d in [d12,d2,d3]],marker_color='#8cc1e6'),secondary_y=True)

fig.update(layout={
    'barmode':'group',
    'xaxis':{'title':'Num of vehicles arrived', 'showline':True},
    'yaxis':{'title': 'Carbon emission(kg)'},
    'yaxis2':{'title': 'Energy delivery(%)'},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.1,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':900,
    'height':450
})
fig.show()


In [ ]:
d1 = pd.read_csv('MPC_offline.csv', index_col=0)
d11 = d1[d1['factor']==1]
d12 = d1[d1['factor']==1.5]
d13 = pd.read_csv('online_P120.csv', index_col=0)
d13 = d13[d13['factor']==1.5]
d2 = pd.read_csv('EDF.csv', index_col=0)
d3 = pd.read_csv('ES.csv', index_col=0)
d4 = pd.read_csv('TOU_P120.csv', index_col=0)
d4 = d4[d4['factor']==1.5]

x = ['Offline','Online', 'TOU', 'EDF','ES']
fig = go.Figure(
    data=[
        go.Bar(name='Total carbon emission (kg)', x=x, y=[np.sum(d['carbon_emission(kg)']) for d in [d12,d13,d4,d2,d3]], yaxis='y', offsetgroup=1,marker_color='#ef857e'),
        go.Bar(name='Energy delivery (%)', x=x, y=[np.sum(d['delivery'])/np.sum(d['required'])*100 for d in [d12,d13,d4,d2,d3]], yaxis='y2', offsetgroup=2,marker_color='#8cc1e6')
    ],
    layout={
        'yaxis': {'title': 'carbon emission'},
        'yaxis2': {'title': 'energy delivery', 'overlaying': 'y', 'side': 'right'}
    }
)

# Change the bar mode
fig.update_layout(barmode='group',
                  width = 900,
                  height= 450,
                  font= {"size": 20},
    template='simple_white',
    legend={'yanchor':'bottom','y':1.1,'xanchor':'left','x':0.0,'orientation':'h'},
                
                )
fig.show()


In [ ]:
5/8*12

# plot Seasonal figures

In [ ]:
spring = [d for d in date if int(str(d)[4:6]) in [3,4,5] ]
spring = [idx for idx, d in enumerate(np.unique(date)) if d in spring]
summer = [d for d in date if int(str(d)[4:6]) in [6,7,8] ]
summer = [idx for idx, d in enumerate(np.unique(date)) if d in summer]
autumn = [d for d in date if int(str(d)[4:6]) in [9,10,11] ]
autumn = [idx for idx, d in enumerate(np.unique(date)) if d in autumn]
winter = [d for d in date if int(str(d)[4:6]) in [12,1,2] ]
winter = [idx for idx, d in enumerate(np.unique(date)) if d in winter]

In [ ]:
data = np.zeros(6742)
with open('../data/Processed Data/CAISO_2021_Daily_CO2_Intensity.csv', newline='') as file:
        reader = csv.reader(file)
        next(reader)
        data = [data for data in reader]
        data_array = np.asarray(data, dtype=float)
carbon_intensity = data_array

In [131]:
spring

[53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141]

In [ ]:
df = np.load(f'../result/season/edf_day{day}_P120.npz')
np.shape(df['carbon'])

In [ ]:
def s2s(s):
    news = np.zeros(24)
    for i in range(287):
        news[i//12] += s[i]
    return news/12

In [130]:
springdays = spring[:10]
summerdays = summer[:10]
autumndays = autumn[:10]
winterdays = winter[:10]
edf1 = np.zeros(287)
edf11 = np.zeros(287)
edf2 = np.zeros(287)
edf21 = np.zeros(287)
edf3 = np.zeros(287)
edf31 = np.zeros(287)
edf4 = np.zeros(287)
edf41 = np.zeros(287)
off1 = np.zeros(287)
off11 = np.zeros(287)
off2 = np.zeros(287)
off21 = np.zeros(287)
off3 = np.zeros(287)
off31 = np.zeros(287)
off4 = np.zeros(287)
off41 = np.zeros(287)
for day in springdays:
    df = np.load(f'../result/season/edf_day{day}_P120.npz')
    edf1 += np.sum(df['carbon'],axis=1)
    edf11 += np.sum(df['u'].T,axis=1)
    df = np.load(f'../result/season/offline_day{day}_P120.npz')
    off1 += np.sum(df['carbon'],axis=1)
    off11 += np.sum(df['u'].T,axis=1)
for day in summerdays:
    df = np.load(f'../result/season/edf_day{day}_P120.npz')
    edf2 += np.sum(df['carbon'],axis=1)
    edf21 += np.sum(df['u'].T,axis=1)
    df = np.load(f'../result/season/offline_day{day}_P120.npz')
    off2 += np.sum(df['carbon'],axis=1)
    off21 += np.sum(df['u'].T,axis=1)
for day in autumndays:
    df = np.load(f'../result/season/edf_day{day}_P120.npz')
    edf3 += np.sum(df['carbon'],axis=1)
    edf31 += np.sum(df['u'].T,axis=1)
    df = np.load(f'../result/season/offline_day{day}_P120.npz')
    off3 += np.sum(df['carbon'],axis=1)
    off31 += np.sum(df['u'].T,axis=1)
for day in winterdays:
    df = np.load(f'../result/season/edf_day{day}_P120.npz')
    edf4 += np.sum(df['carbon'],axis=1)
    edf41 += np.sum(df['u'].T,axis=1)
    df = np.load(f'../result/season/offline_day{day}_P120.npz')
    off4 += np.sum(df['carbon'],axis=1)
    off41 += np.sum(df['u'].T,axis=1)

In [ ]:
s2s(edf1)

In [ ]:
fig = make_subplots(rows=3, cols=4, subplot_titles=("Spring", "Summer", "Autumn", "Winter"))

x = list(range(24))
fig.add_trace(go.Scatter(name='Carbon intensity', x=x, y=s2s(carbon_intensity[spring[0]][1:]), marker_color='#ffa05a',line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(name='Carbon emission: baseline', x=x, y=s2s(edf1),line={'width':3,'color':'#8dbf79'}),row=2, col=1)
fig.add_trace(go.Scatter(name='Carbon emission: carbon aware', x=x, y=s2s(off1),line={'width':3,'color':'#ef857e',}),row=2, col=1)
fig.add_trace(go.Scatter(name='Charging schedule: baseline', x=x, y=s2s(edf11),line={'width':3,'color':'#b076de'}),row=3, col=1)
fig.add_trace(go.Scatter(name='Charging schedule: carbon aware', x=x, y=s2s(off11),line={'width':3,'color':'#8cc1e6',}),row=3, col=1)


fig.add_trace(go.Scatter(name='Carbon intensity', x=x, y=s2s(carbon_intensity[summer[0]][1:]), marker_color='#ffa05a',line={'width':3},showlegend=False),row=1, col=2)
fig.add_trace(go.Scatter(name='Carbon emission: baseline',showlegend=False, x=x, y=s2s(edf2),line={'width':3,'color':'#8dbf79'}),row=2, col=2)
fig.add_trace(go.Scatter(name='Carbon emission: carbon aware',showlegend=False, x=x, y=s2s(off2),line={'width':3,'color':'#ef857e'}),row=2, col=2)
fig.add_trace(go.Scatter(name='Charging schedule: baseline',showlegend=False, x=x, y=s2s(edf21),line={'width':3,'color':'#b076de'}),row=3, col=2)
fig.add_trace(go.Scatter(name='Charging schedule: carbon aware', showlegend=False,x=x, y=s2s(off21),line={'width':3,'color':'#8cc1e6',}),row=3, col=2)

fig.add_trace(go.Scatter(name='Carbon intensity', x=x, y=s2s(carbon_intensity[autumn[0]][1:]), marker_color='#ffa05a',line={'width':3},showlegend=False),row=1, col=3)
fig.add_trace(go.Scatter(name='Carbon emission: baseline',showlegend=False, x=x, y=s2s(edf3),line={'width':3,'color':'#8dbf79'}),row=2, col=3)
fig.add_trace(go.Scatter(name='Carbon emission: carbon aware',showlegend=False, x=x, y=s2s(off3),line={'width':3,'color':'#ef857e'}),row=2, col=3)
fig.add_trace(go.Scatter(name='Charging schedule: baseline', showlegend=False,x=x, y=s2s(edf31),line={'width':3,'color':'#b076de'}),row=3, col=3)
fig.add_trace(go.Scatter(name='Charging schedule: carbon aware',showlegend=False, x=x, y=s2s(off31),line={'width':3,'color':'#8cc1e6',}),row=3, col=3)

fig.add_trace(go.Scatter(name='Carbon intensity', x=x, y=s2s(carbon_intensity[winter[0]][1:]), marker_color='#ffa05a',line={'width':3},showlegend=False),row=1, col=4)
fig.add_trace(go.Scatter(name='Carbon emission: baseline', showlegend=False,x=x, y=s2s(edf4),line={'width':3,'color':'#8dbf79'}),row=2, col=4)
fig.add_trace(go.Scatter(name='Carbon emission: carbon aware', showlegend=False,x=x, y=s2s(off4),line={'width':3,'color':'#ef857e'}),row=2, col=4)
fig.add_trace(go.Scatter(name='Charging schedule: baseline', showlegend=False,x=x, y=s2s(edf41),line={'width':3,'color':'#b076de'}),row=3, col=4)
fig.add_trace(go.Scatter(name='Charging schedule: carbon aware',showlegend=False, x=x, y=s2s(off41),line={'width':3,'color':'#8cc1e6',}),row=3, col=4)

fig.update(layout={
    'xaxis9':{'title':'Time (Hours)', 'showline':True},
    'yaxis':{'title': 'C(t)'},
    'yaxis5':{'title': 'Carbo(kg)'},
    'yaxis9':{'title': 'u(t)'},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':1500,
    'height':700
})
fig.update_annotations(font_size=20)
fig.show()


In [ ]:
fig.write_image("image/season2_P120.pdf")

In [ ]:
mpc_offline = pd.read_csv('MPC_offline.csv', index_col=0)
mpc_offline = mpc_offline[mpc_offline['factor']==1]
mpc_offline = mpc_offline[mpc_offline['num_vehicle'] < 25]
edf = pd.read_csv('EDF.csv', index_col=0)
edf = edf[edf['num_vehicle'] < 25]

print(len(edf), len(mpc_offline))

mpc_offline_spring = mpc_offline.loc[mpc_offline['day'].isin(spring)]
edf_spring = edf.loc[edf['day'].isin(spring)]
print(len(edf_spring), len(mpc_offline_spring))

mpc_offline_2 = mpc_offline.loc[mpc_offline['day'].isin(summer)]
edf_2 = edf.loc[edf['day'].isin(summer)]
print(len(edf_2), len(mpc_offline_2))

mpc_offline_3 = mpc_offline.loc[mpc_offline['day'].isin(autumn)]
edf_3 = edf.loc[edf['day'].isin(autumn)]
print(len(edf_3), len(mpc_offline_3))

mpc_offline_4 = mpc_offline.loc[mpc_offline['day'].isin(winter)]
edf_4 = edf.loc[edf['day'].isin(winter)]
print(len(edf_4), len(mpc_offline_4))

In [ ]:
edf_spring_carbon = np.sum(edf_spring['carbon_emission(kg)'])
mpc_offline_spring_carbon = np.sum(mpc_offline_spring['carbon_emission(kg)'])
edf_spring_energy = np.sum(edf_spring['delivery']) / np.sum(edf_spring['required']) *100
mpc_offline_spring_energy = np.sum(mpc_offline_spring['delivery']) / np.sum(mpc_offline_spring['required']) *100

edf2_carbon = np.sum(edf_2['carbon_emission(kg)'])
mpc_offline2_carbon = np.sum(mpc_offline_2['carbon_emission(kg)'])
edf2_energy = np.sum(edf_2['delivery']) / np.sum(edf_2['required']) *100
mpc_offline2_energy = np.sum(mpc_offline_2['delivery']) / np.sum(mpc_offline_2['required']) *100

edf3_carbon = np.sum(edf_3['carbon_emission(kg)'])
mpc_offline3_carbon = np.sum(mpc_offline_3['carbon_emission(kg)'])
edf3_energy = np.sum(edf_3['delivery']) / np.sum(edf_3['required']) *100
mpc_offline3_energy = np.sum(mpc_offline_3['delivery']) / np.sum(mpc_offline_3['required']) *100

edf4_carbon = np.sum(edf_4['carbon_emission(kg)'])
mpc_offline4_carbon = np.sum(mpc_offline_4['carbon_emission(kg)'])
edf4_energy = np.sum(edf_4['delivery']) / np.sum(edf_4['required']) *100
mpc_offline4_energy = np.sum(mpc_offline_4['delivery']) / np.sum(mpc_offline_4['required']) *100


In [ ]:
from plotly.subplots import make_subplots
x = ['carbon-aware','baseline']
fig = make_subplots(rows=2, cols=2, subplot_titles=("Spring", "Summer", "Autumn", "Winter"),specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]])


fig.add_trace(go.Bar(name='Total carbon_emission(kg)', x=x, y=[mpc_offline_spring_carbon,edf_spring_carbon],marker_color='#ef857e'),row=1, col=1,secondary_y=False)
fig.add_trace(go.Scatter(name='Energy delivery(%)', x=x, y=[mpc_offline_spring_energy,edf_spring_energy], marker_color='#8cc1e6',line={'width':4}),row=1, col=1, secondary_y=True)

fig.add_trace(go.Bar(name='carbon_emission(kg)', x=x, y=[mpc_offline2_carbon,edf2_carbon],marker_color='#ef857e',showlegend=False),row=1, col=2)
fig.add_trace(go.Scatter(name='Energy delivery(%)', x=x, y=[mpc_offline2_energy,edf2_energy], marker_color='#8cc1e6',showlegend=False,line={'width':4}),row=1, col=2, secondary_y=True)

fig.add_trace(go.Bar(name='carbon_emission(kg)', x=x, y=[mpc_offline3_carbon,edf3_carbon],marker_color='#ef857e',showlegend=False),row=2, col=1)
fig.add_trace(go.Scatter(name='Energy delivery(%)', x=x, y=[mpc_offline3_energy,edf3_energy], marker_color='#8cc1e6',showlegend=False,line={'width':4}),row=2, col=1, secondary_y=True)

fig.add_trace(go.Bar(name='carbon_emission(kg)', x=x, y=[mpc_offline4_carbon,edf4_carbon],marker_color='#ef857e',showlegend=False),row=2, col=2)
fig.add_trace(go.Scatter(name='Energy delivery(%)', x=x, y=[mpc_offline4_energy,edf4_energy], marker_color='#8cc1e6',showlegend=False,line={'width':4}),row=2, col=2, secondary_y=True)

fig.update_layout(height=800, width=900,template='simple_white',yaxis2={'range':[1,101]},yaxis4={'range':[1,101]},yaxis6={'range':[1,101]},yaxis8={'range':[1,101]},legend={'yanchor':'bottom','y':1.1,'xanchor':'left','x':-0.05,'orientation':'h'},font={"size": 20})
fig.show()

In [ ]:
fig.write_image("image/season.pdf")

# plot EV figures

In [ ]:
total_num_of_data_entries = 6742
arrival_time = np.zeros(total_num_of_data_entries)
arrival_time2 = np.zeros(24)
arrival_time3 = np.zeros(24)
departure_time = np.zeros(total_num_of_data_entries)
date = np.zeros(total_num_of_data_entries)
with open('../BerkeleyDataCleaned_timestep_adjusted.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)

    line_count=0
    for row in reader:
        theRow = list(row)
        arrival_time[line_count] = float(theRow[2])
        arrival_time2[int(theRow[2])//12] += 1
        departure_time[line_count] = float(theRow[3])
        if theRow[0] == '20210715':
            arrival_time3[int(theRow[2])//12] += 1
        date[line_count]=(float(theRow[0]))
        line_count += 1

arrival_time, departure_time

In [ ]:
int(theRow[2])//12

In [ ]:
fig = go.Figure()
#fig.add_trace(go.Histogram(x=arrival_time,name='Number of vehicles'))
fig.add_trace(go.Bar(y=arrival_time2))
#fig.add_traces(go.Scatter(x=list(range(24)),y=np.mean(carbon_intensity,axis=0)[1:],name='Carbon intensity',yaxis='y2'))

fig.update_layout(template='simple_white',xaxis_title='Time (Hours)', yaxis_title='Number of vehicles',width=700,height=400, yaxis2={'title': 'Carbon intensity', 'overlaying': 'y', 'side': 'right','range':[-0,0.5]},legend={'yanchor':'bottom','y':1.1,'xanchor':'left','x':-0.05,'orientation':'h'},font={"size": 20})
fig.show()

In [ ]:
fig.write_image("image/arrival_time_distribution.pdf")

In [ ]:
fig = make_subplots(rows=1, cols=2,subplot_titles=("All days in 2021","July 15th, 2021"))
fig.add_trace(go.Bar(y=arrival_time2,marker_color='#2077b4',showlegend=False),row=1,col=1)
fig.add_trace(go.Bar(y=arrival_time3,marker_color='#2077b4',showlegend=False),row=1,col=2)

fig.update_layout(template='simple_white',xaxis_title='Time (Hours)', yaxis_title='Total vehicles arrived',font={"size": 20},title_font={"size": 20},width=1000,height=500,xaxis2={'title':'Time (Hours)'})
fig.update_annotations(font_size=22)
fig.show()


In [ ]:
fig.write_image("image/arrival_time_distribution.pdf")

In [ ]:
res = []
starting_index=0
end_index=0
for current_date in range(336):
    starting_index=np.copy(end_index)
    num_of_vehicles=0

    while date[end_index]==date[starting_index]:
        num_of_vehicles+=1
        end_index+=1

    res.append(num_of_vehicles)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=res, name='solution',xbins=dict(start=0.0, end=50, size=1)))
fig.update_layout(template='simple_white',xaxis_title='Number of vehicles', yaxis_title='Number of days',width=900,height=500,font=dict({"size": 20}))
fig.show()

In [ ]:
fig.write_image("image/distribution_numvehicle.pdf")

In [ ]:
unique, counts = np.unique(np.array(res), return_counts=True)
dict(zip(unique, counts))
